## Bayesian methods of hyperparameter optimization

This kernel is about using library BayesianOptimization, that can do parameters tuning for us much easier. This library has very good documentation, so I will use information from this and you can find there much more information.

Documentation:
https://github.com/fmfn/BayesianOptimization

At first this is simple data preparation to show, how to work with library. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [2]:
train_df = pd.read_csv('../input/flight_delays_train.csv')
test_df = pd.read_csv('../input/flight_delays_test.csv')
train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [3]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

In [4]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])

X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

In [6]:
full_df

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,end_of_month,hour,morning,day,evening,night,winter,spring,summer,autumn,holiday,weekday,airport_dest_per_month,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,1,19,0,0,1,0,0,0,1,0,1,0,746,1016,8290,11375,18024,1569,3.436597e-01,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,1,15,0,1,0,0,0,1,0,0,0,1,313,105,3523,1390,13069,1094,-6.129071e-01,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,0,14,0,1,0,0,0,0,0,1,1,0,166,136,2246,1747,11737,977,-8.358074e-01,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,1,10,1,0,0,0,0,0,0,1,1,0,136,514,1785,6222,15343,1242,-8.849876e-01,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,0,18,0,1,0,0,0,0,0,1,1,0,48,226,687,2571,30958,2674,7.323820e-02,-0.997314,5144,1313,1103
5,8,3,4,14,184,180,683,3045,1,0,0,19,0,0,1,0,0,0,1,0,0,1,308,169,3523,1824,12005,1022,3.040331e-01,-0.952661,5252,1081,1115
6,1,27,4,7,217,166,1035,3752,0,0,1,7,1,0,0,0,1,0,0,0,0,1,287,78,3356,768,14624,1291,-3.923371e-01,0.919821,6437,1000,1354
7,4,29,6,15,201,74,596,3278,0,0,1,6,0,0,0,1,0,1,0,0,1,0,329,324,3903,3835,7697,652,-9.150162e-02,0.995805,5588,427,1211
8,7,28,5,1,212,82,1189,3542,0,0,1,7,1,0,0,0,0,0,1,0,1,0,725,93,8290,1096,18024,1542,-3.461171e-01,0.938191,5993,494,1306
9,6,20,2,16,81,230,853,1325,0,0,1,20,0,0,1,0,0,0,1,0,0,1,6,576,99,6222,15343,1278,5.642467e-01,-0.825606,2304,1469,484


Now we have data to tune parameters for different models.

In [7]:
X_train.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,end_of_month,hour,morning,day,evening,night,winter,spring,summer,autumn,holiday,weekday,airport_dest_per_month,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,1,19,82,732,171,0,0,1,19,0,0,1,0,0,0,1,0,1,0,746,1016,8290,11375,18024,1569,0.343660,-0.939094,265,494,67
1,4,20,3,19,226,180,834,3986,0,0,1,15,0,1,0,0,0,1,0,0,0,1,313,105,3523,1390,13069,1094,-0.612907,-0.790155,6907,1085,1441
2,9,2,5,21,239,62,416,4091,1,0,0,14,0,1,0,0,0,0,0,1,1,0,166,136,2246,1747,11737,977,-0.835807,-0.549023,7064,359,1518
3,11,25,6,16,81,184,872,1304,0,0,1,10,1,0,0,0,0,0,0,1,1,0,136,514,1785,6222,15343,1242,-0.884988,0.465615,2258,1122,484
4,10,7,6,20,182,210,423,2979,1,0,0,18,0,1,0,0,0,0,0,1,1,0,48,226,687,2571,30958,2674,0.073238,-0.997314,5144,1313,1103


## How does it work

Bayesian optimization works by constructing a posterior distribution of functions (gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation taking into account what it knows about the target function. At each step a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with a exploration strategy (such as UCB (Upper Confidence Bound), or EI (Expected Improvement)), are used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Simple example

At first you should create an optimizer. It uses two things:
* function to optimize
* bounds of parameters

For us function is the procedure, which counts metrics of our model quality.

**!** The important thing is that our optimization will maximize the value on function. So if your metric should be smaller the better, don't forget to use negative metric value.

In [8]:
def simple_functon(a, b):
    return a + b

In [9]:
optimizer = BayesianOptimization(
    simple_functon,
    {'a': (1, 3),
    'b': (4, 7)})

Main parameters of this function:

* **n_iter**: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
* **init_points**: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

In [10]:
optimizer.maximize(3, 2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        |  8.115    |  1.916    |  6.199    |
|  2        |  6.198    |  1.544    |  4.654    |
|  3        |  7.021    |  2.971    |  4.05     |
|  4        |  8.965    |  2.013    |  6.952    |
|  5        |  10.0     |  3.0      |  7.0      |


Ideal! We can see the best params:

In [11]:
optimizer.max['params']

{'a': 3.0, 'b': 7.0}

... and the best result

In [12]:
optimizer.max['target']

10.0

**!** The important thing is that our optimization will maximize the value on function. So if your metric should be smaller the better, don't forget to use negative metric value. Optimizer use float values of params, you should use int() in function, if this parameter must be integer.

## Test it on data

### LigthGBM

My kernel about using it on real data and real peremeters with LightGBM: 
https://www.kaggle.com/clair14/gold-is-the-reason-teams-and-bayes-for-lightgbm

There I will use random values of parameters to test.

In [13]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

This is function, that we want to maximize - function, that counts cross-validation metrics of lightGBM for our params.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [14]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=3)
    return cv_result['auc-mean'][-1]

In [15]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.7213   |  0.002125 |  0.04664  |  41.43    |  4.167e+0 |  487.7    |  3.497e+0 |
|  2        |  0.746    |  0.04656  |  0.02866  |  19.28    |  3.722e+0 |  1.775e+0 |  760.0    |
|  3        |  0.7462   |  0.04308  |  0.02692  |  41.47    |  9.999e+0 |  1.894e+0 |  148.2    |
|  4        |  0.7149   |  0.00658  |  0.04714  |  36.02    |  102.8    |  140.9    |  35.88    |
|  5        |  0.7455   |  0.0384   |  0.000793 |  58.45    |  82.42    |  1.984e+0 |  3.876e+0 |
|  6        |  0.7455   |  0.02062  |  0.04439  |  44.49    |  9.827e+0 |  1.998e+0 |  3.97e+03 |
|  7        |  0.7458   |  0.03947  |  0.04974  |  60.74    |  5.65e+03 |  1.998e+0 |  71.24    |
|  8        |  0.7454   |  0.0311   |  0.03607  |  26.08    |  108.9    |  1.994e+0 |  107.9    |
|  9        |  0.744

Now you can see the result

In [16]:
lgbBO.max

{'target': 0.7461538363671112,
 'params': {'lambda_l1': 0.043076199306701035,
  'lambda_l2': 0.02691844694082331,
  'max_depth': 41.472336297810536,
  'min_child_samples': 9999.389460233944,
  'min_data_in_leaf': 1894.4839005465249,
  'num_leaves': 148.18331798686245}}

And all the process in each step...

In [17]:
lgbBO.res[0]

{'target': 0.721317547274817,
 'params': {'lambda_l1': 0.002125056871858094,
  'lambda_l2': 0.04664067569564421,
  'max_depth': 41.433496363147505,
  'min_child_samples': 4167.296765142901,
  'min_data_in_leaf': 487.6902406865694,
  'num_leaves': 3496.843147731591}}

## Loading progress

It is wonderful! Really! You can learn you optimizer, collect some points, then you can correct something (bounds, for example, if you understand, that some values are not interesting for you. There is no point to start from beginning, you can just use previous result)
May be we can change data just a little bit, and continue to search for best parameters.

In [18]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path="./logs.json")
lgbBO.subscribe(Events.OPTMIZATION_STEP, logger)

In [19]:
lgbBO.maximize(n_iter=10, init_points=3)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  13       |  0.7295   |  0.012    |  0.02476  |  40.37    |  9.179e+0 |  860.5    |  379.0    |
|  14       |  0.728    |  0.04683  |  0.02356  |  32.16    |  5.169e+0 |  720.4    |  463.0    |
|  15       |  0.7357   |  0.002992 |  0.04203  |  45.93    |  2.647e+0 |  1.103e+0 |  2.81e+03 |
|  16       |  0.7457   |  0.03453  |  0.008974 |  57.12    |  9.994e+0 |  1.982e+0 |  1.532e+0 |
|  17       |  0.7459   |  0.0176   |  0.04975  |  62.27    |  3.056e+0 |  1.992e+0 |  3.957e+0 |
|  18       |  0.7456   |  0.01966  |  0.02828  |  56.0     |  2.195e+0 |  1.997e+0 |  1.813e+0 |
|  19       |  0.745    |  0.04923  |  0.00299  |  7.422    |  275.2    |  1.998e+0 |  3.657e+0 |
|  20       |  0.7461   |  0.00661  |  0.006395 |  62.77    |  6.804e+0 |  1.995e+0 |  863.9    |
|  21       |  0.745

Now we can read it for another optimizer

In [20]:
new_opt = BayesianOptimization(lgb_eval, {'num_leaves': (25, 100),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 100),
                                                'min_data_in_leaf': (50, 200)
                                                })

In [21]:
from bayes_opt.util import load_logs

load_logs(new_opt, logs=["./logs.json"]);

In [22]:
new_opt.maximize(n_iter=5, init_points=1)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.7154   |  0.03215  |  0.01448  |  12.09    |  86.35    |  102.3    |  34.7     |
|  2        |  0.7164   |  0.04695  |  0.02481  |  62.69    |  96.45    |  199.2    |  96.25    |
|  3        |  0.7165   |  0.006139 |  0.02536  |  62.82    |  86.64    |  198.0    |  95.78    |
|  4        |  0.7164   |  0.03107  |  0.0186   |  62.28    |  92.21    |  199.0    |  98.72    |
|  5        |  0.7165   |  0.02256  |  0.01627  |  62.27    |  83.26    |  199.6    |  94.54    |
|  6        |  0.7174   |  0.02339  |  0.03609  |  5.167    |  89.69    |  199.5    |  98.98    |


In [23]:
new_opt.max

{'target': 0.7460840974007402,
 'params': {'lambda_l1': 0.006609676369548107,
  'lambda_l2': 0.0063945366479056355,
  'max_depth': 62.77108278289809,
  'min_child_samples': 6803.675955317543,
  'min_data_in_leaf': 1994.6549460420729,
  'num_leaves': 863.8529511064605}}

As you can see this is point from the previous run

## Try certain points

We can choose the certain point ant try the result. 

In [24]:
new_opt.probe({'num_leaves': 10,
                'max_depth': 100,
                'lambda_l2': 1,
                'lambda_l1': 1,
                'min_child_samples': 300,
                'min_data_in_leaf': 1000 })

new_opt.probe({'num_leaves': 55,
                'max_depth': 400,
                'lambda_l2': 5,
                'lambda_l1': 5,
                'min_child_samples': 100,
                'min_data_in_leaf': 10 })

In [25]:
new_opt.maximize(n_iter=0, init_points=0)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
|  7        |  0.7301   |  1.0      |  1.0      |  100.0    |  300.0    |  1e+03    |  10.0     |
|  8        |  0.7149   |  5.0      |  5.0      |  400.0    |  100.0    |  10.0     |  55.0     |


### CatBoost

Let's try another model for test

In [26]:
def cat_eval(num_leaves,max_depth,bagging_temperature, l2_leaf_reg):
    params = {'bagging_temperature': bagging_temperature,
              'num_leaves': int(num_leaves),
              'max_depth': int(max_depth),
              'l2_leaf_reg': l2_leaf_reg,
              'iterations': 500,
              'learning_rate':0.1,
              'early_stopping_rounds':100,
              'eval_metric': "AUC",
              'verbose': False}
    cv_dataset = Pool(data=X_train,
                  label=y_train,
                  cat_features=categorical_features)
    scores = cv(cv_dataset,
            params,
            fold_count=3)
    return scores['test-AUC-mean'].max()

In [27]:
cat_opt = BayesianOptimization(cat_eval, {'num_leaves': (25, 100),
                                          'max_depth': (5, 15),
                                          'bagging_temperature': (0.1, 0.9),
                                          'l2_leaf_reg': (2,5)
                                        })

In [28]:
cat_opt.maximize(n_iter=5, init_points=2)

|   iter    |  target   | baggin... | l2_lea... | max_depth | num_le... |
-------------------------------------------------------------------------


KeyboardInterrupt: 

## Сonclusion

This library shows perfect results, and it is much effective then rendom search or CV gread, as you don't need to try every point.

Thanks for your attention!

<img src='https://sites.google.com/site/bayesforvietnam/_/rsrc/1465811460099/home/Bayes%201.jpg'/>